In [ ]:
%%capture

!apt-get update
!apt-get install -y openjdk-8-jdk-headless -qq
!apt-get install maven -qq

!curl -L "https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz" > spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark py4j
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql import functions
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
#from pyspark.sql.Column import isNull
from pyspark.sql.functions import lit
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "16g").getOrCreate()

In [15]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit
from pyspark.sql.functions import sum,avg,max,min,mean,count
spark=SparkSession.builder.appName("Spark Dataframes").getOrCreate()



In [28]:
# SPARK UDF are like custom fxn which we can map on DF , like we used to do on RDD for this import the belwo
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType   # as udf fill return integer




In [26]:
df=spark.read.options(header='True',inferSchema='True').csv('OfficeData.csv')
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [29]:
def get_total_salary(salary,bonus):   # fxn for UDF
  return salary + bonus

totalSalaryUDF=udf(lambda x,y:get_total_salary(x,y),IntegerType())   # define UDF and register fxn

df.withColumn("total_salary",totalSalaryUDF(df.salary,col("bonus"))).show()  # passing column to udf

+-------------+----------+-----+------+---+-----+------------+
|employee_name|department|state|salary|age|bonus|total_salary|
+-------------+----------+-----+------+---+-----+------------+
|        James|     Sales|   NY| 90000| 34|10000|      100000|
|      Michael|     Sales|   NY| 86000| 56|20000|      106000|
|       Robert|     Sales|   CA| 81000| 30|23000|      104000|
|        Maria|   Finance|   CA| 90000| 24|23000|      113000|
|        Raman|   Finance|   CA| 99000| 40|24000|      123000|
|        Scott|   Finance|   NY| 83000| 36|19000|      102000|
|          Jen|   Finance|   NY| 79000| 53|15000|       94000|
|         Jeff| Marketing|   CA| 80000| 25|18000|       98000|
|        Kumar| Marketing|   NY| 91000| 50|21000|      112000|
+-------------+----------+-----+------+---+-----+------------+

